In [7]:
import pandas as pd

visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

# 1. We inspect the dfs using print and head:

#print(visits.head())
#print(cart.head())
#print(checkout.head())
#print(purchase.head())

# After inspecting the dfs, we verify that there are no duplicate user ids in the df visits. 

#print(len(visits)) # 2000
#print(visits.user_id.nunique()) # 2000

# Since there are 2000 unique users out of 2000 rows, we know that there are no duplicate 
# ids in the visits df. This is important since using len(visits) will match the number of 
# individual users in this funnel.

# We verify duplicate user ids in the df cart.

#print(len(cart)) # 400
#print(cart.user_id.nunique()) # 348

# This discrepancy shows that although there are 400 rows in the this df, 
# only 348 individual users are represented, indicating that there must be duplicate 
# users under the user_id column.

# We verify duplicate user ids in the df checkout

#print(len(checkout)) # 360
#print(checkout.user_id.nunique()) # 226

# We verify duplicate user ids in the df purchase

#print(len(purchase)) # 252
#print(purchase.user_id.nunique()) # 144

# Since 3 out of 4 dfs contain duplicate user ids, we must construct new dfs that only account 
# for each individual user id, in order to have accurate analytics involving our funnel. 
# The df visits will not need to be re-constructed.

cart.drop_duplicates(subset='user_id', keep='first', inplace=True)
cart.reset_index(inplace=True, drop=True)
#print(len(cart)) # the number is now 348

checkout.drop_duplicates(subset='user_id', keep='first', inplace=True)
checkout.reset_index(inplace=True, drop=True)
#print(len(checkout)) # the number is now 226

purchase.drop_duplicates(subset='user_id', keep='first', inplace=True)
purchase.reset_index(inplace=True, drop=True)
#print(len(purchase)) # the number is now 144

# 2. We combine visits and cart using a left merge

visits_cart = pd.merge(
    visits,
    cart,
    how='left')
#print(visits_cart)

# 3. How long is our merged DataFrame?

print(len(visits_cart))
# [2000 rows x 3 columns]

# 4. How many of the timestamps are null for the column cart_time?

# My original code:

visits_count = visits_cart.visit_time.count()
cart_time_count = visits_cart.cart_time.count()
cart_time_null_count = visits_count - cart_time_count
#print(cart_time_null_count) # 1652

# More elegant code suggested by CA:

cart_time_null_count = visits_cart[visits_cart.cart_time.isnull()]
#print(cart_time_null_count)
print(len(cart_time_null_count))
# [1652 rows x 3 columns]

# 1652 timestamps are null for the column cart_time, 
# meaning that out of all the people who visited the page (2000), 
# 1652 of them did not add anything to a cart.

# 5. What percent of users who visited Cool T-Shirts Inc. ended up not placing a t-shirt in their cart?

percent_visits_not_cart = (float(len(cart_time_null_count)) / float(len(visits_cart))) * 100
print(percent_visits_not_cart) # 82.6%

# 6. We repeat the left merge for cart and checkout and count null values. 
# What percentage of users put items in their cart, but did not proceed to checkout?

cart_checkout = pd.merge(
    cart,
    checkout,
    how='left')
#print(cart_checkout)

checkout_time_null_count = cart_checkout[cart_checkout.checkout_time.isnull()]
#print(checkout_time_null_count)
#print(len(checkout_time_null_count)) # 122

percent_cart_not_checkout = (float(len(checkout_time_null_count)) / float(len(cart_checkout))) * 100 
print(percent_cart_not_checkout) # 35.06%

# 7. We merge all four steps of the funnel, in order, using a series of left merges. 
# We save the results to the variable all_data. We examine the result using print and head.

all_data = visits.merge(cart, how='left').merge(checkout, how='left').merge(purchase, how='left')
#print(all_data.head())
#print(len(all_data))

# 8. What percentage of users proceeded to checkout, but did not purchase a t-shirt?

checkout_purchase = pd.merge(
    checkout, 
    purchase, 
    how='left')
#print(checkout_purchase)
#print(len(checkout_purchase)) # 226

purchase_time_null_count = checkout_purchase[checkout_purchase.purchase_time.isnull()]
#print(purchase_time_null_count)
#print(len(purchase_time_null_count)) # 82

percent_checkout_not_purchase = (float(len(purchase_time_null_count)) / float(len(checkout_purchase))) * 100
print(percent_checkout_not_purchase) # 36.28%

# 9. Which step of the funnel is weakest (i.e., has the highest percentage of users not completing it)? 
# How might Cool T-Shirts Inc. change their website to fix this problem?

# percent_visits_not_cart = 82.6%
# percent_cart_not_checkout = 35.06%
# percent_checkout_not_purchase = 36.28%

# The weakest step of the funnel is from users initially visiting 
# to actually placing an item in their cart.
# Fixes: the "add to cart" button can be made more conspicuous by changing 
# it's color to stand out more; pre-highlighting the button so users see it 
# "pre-selected"; placing the "add to cart" button to a place where users are 
# more likely to look, such as the top of the page, if possible; perhaps 
# discounting prices on select items to incentivize visitors to buy more; 
# and using cookies or some type of tracking to offer visitors types of 
# T-shirts they are more likely to buy.

# 10. Using the giant merged DataFrame all_data that we created, 
# let's calculate the average time from initial visit to final purchase.

all_data['time_to_purchase'] = \
    all_data.purchase_time - \
    all_data.visit_time
#print(all_data.head())

# 11. We examine the results using:

#print(all_data.time_to_purchase)

# 12. We calculate the average time to purchase using the following code:

print(all_data.time_to_purchase.mean())
# 0 days 00:42:33.333333

2000
1652
82.6
35.0574712644
36.2831858407
0 days 00:42:33.333333
